# Estatísticas dos parâmetros de um modelo

1. Faça uma função que receba como entrada uma rede neural e retorne um dicionário no qual cada chave é o nome de um parâmetro do modelo e os respectivos valores são:
    * O maior e menor valor do parâmetro;
    * O maior e menor valor do gradiente do parâmetro;
2. Faça uma função que receba o dicionário de 1 e retorne o menor e maior valor dentre todos os parâmetros e todos os gradientes (4 valores no total);
2. Modifique o script de treinamento visto nas aulas para plotar além da loss e da acurácia, os menores e maiores valores retornados em 2. Crie um terceiro gráfico (além do gráfico da loss e acurácia), para mostrar os valores. Os valores devem ser plotados **durante o treinamento da rede**
    

Exemplo

In [1]:
import torch
from torch import nn
from torchvision import models

# Imagem aleatória para ilustração
img = torch.rand(1, 3, 224, 224)
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

Exemplo

In [30]:
def max_grad(model):
    """Máximo valor de gradiente de cada parâmetro."""

    stats = {}
    # model.named_parameters() retorna os nomes e respectivos parâmetros do
    # modelo
    for param_name, param in model.named_parameters():
        grad = param.grad
        g_max = grad.max()
        stats[param_name] = {'g_max':g_max.item()}

    return stats

# Aplica o modelo
scores = model(img)
# Calcula os gradientes (em um treinamento real seria loss.backward())
scores.sum().backward()

#max_grad(model)

### Resolução 

**1.** Função que recebe como entrada uma rede neural e retorna um dicionário contendo como chave o nome do parametro do modelo e o maior e menor valor dos parâmetros, bem como dos gradientes dos parêmetros.

In [14]:
import json

def param_stats(model):
    """Retorna um dicionário com estatísticas de cada parâmetro do modelo."""

    stats = {}
    for param_name, param in model.named_parameters():
        param_data = param.data
        grad = param.grad

        stats[param_name] = {
            'param_max': param_data.max().item(),
            'param_min': param_data.min().item(),
            'grad_max': grad.max().item() if grad is not None else None,
            'grad_min': grad.min().item() if grad is not None else None
        }
    # Assim as informacoes ficam na mesma linha, ruim pra ler
    #return stats 
    
    # Organiza as informacoes do dicionario
    return json.dumps(stats, indent=4)


In [15]:
import torch
import torch.nn as nn

# Definindo um exemplo de modelo de rede neural
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Criando uma instância do modelo
model = SimpleNN()

# Exemplo de input
img = torch.randn(1, 1, 28, 28)

# Passando o input pelo modelo para gerar os scores
scores = model(img)

# Calculando os gradientes (em um treinamento real seria loss.backward())
scores.sum().backward()

# Obtendo as estatísticas dos parâmetros e gradientes
stats = param_stats(model)
print(stats)

{
    "fc1.weight": {
        "param_max": 0.03571409359574318,
        "param_min": -0.03571414574980736,
        "grad_max": 1.1916474103927612,
        "grad_min": -1.1999832391738892
    },
    "fc1.bias": {
        "param_max": 0.03552022948861122,
        "param_min": -0.03570165857672691,
        "grad_max": 0.35915958881378174,
        "grad_min": -0.35666465759277344
    },
    "fc2.weight": {
        "param_max": 0.08835775405168533,
        "param_min": -0.08829647302627563,
        "grad_max": 0.9300462007522583,
        "grad_min": 0.0
    },
    "fc2.bias": {
        "param_max": 0.08279874175786972,
        "param_min": -0.06388561427593231,
        "grad_max": 1.0,
        "grad_min": 1.0
    }
}


**2.** Função que recebe o dicionário e retorna o valor máximo e mínimo dos parâmetros e gradientes dos parâmetros

In [16]:
def get_global_stats(stats):
    """Retorna o menor e maior valor dentre todos os parâmetros e gradientes."""
    param_max = float('-inf')
    param_min = float('inf')
    grad_max = float('-inf')
    grad_min = float('inf')

    for param_name, param_stats in stats.items():
        param_max = max(param_max, param_stats['param_max'])
        param_min = min(param_min, param_stats['param_min'])
        if param_stats['grad_max'] is not None:
            grad_max = max(grad_max, param_stats['grad_max'])
        if param_stats['grad_min'] is not None:
            grad_min = min(grad_min, param_stats['grad_min'])

    return {
        'param_max': param_max,
        'param_min': param_min,
        'grad_max': grad_max,
        'grad_min': grad_min
    }


In [17]:
global_stats = get_global_stats(stats)
for key, value in global_stats.items():
    print(f"{key}: {value}")


AttributeError: 'str' object has no attribute 'items'

**3.** 